In [1]:
import sys
sys.path.append("../../../motor-imagery-classification-2024/")

import torch
from torch import nn
import optuna
import torchsummary
from datetime import datetime
import json
import numpy as np

c:\Users\rahar\anaconda3\envs\neuro\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from classification.classifiers import DeepClassifier
from classification.loaders import load_data
from models.unet.eeg_unets import UnetConfig,Unet,BottleNeckClassifier
import lightning as L
from lightning import Fabric

torch.set_float32_matmul_precision('medium')

c:\Users\rahar\anaconda3\envs\neuro\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [3]:
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=20)
sampler = optuna.samplers.TPESampler(seed=10)
study = optuna.create_study(direction="maximize", pruner=pruner,sampler=sampler)

[I 2024-04-18 15:11:42,538] A new study created in memory with name: no-name-144dd24e-4780-471d-b629-cf4fb5374cc6


In [4]:
dataset = {}
for i in range(1,10):
	mat_train,mat_test = load_data("../../data/2b_iv/",i)
	dataset[f"subject_{i}"] = {"train":mat_train,"test":mat_test}

In [5]:
FABRIC = Fabric(accelerator="cuda",precision="bf16-mixed")

Using bfloat16 Automatic Mixed Precision (AMP)


In [6]:
with open(r"params_2024_03_30_12_29.json","r") as f:
	params = json.load(f)

In [7]:
params

{'conv_kernel': 19,
 'max_channels': 256,
 'pool_op': 'avg',
 'starting_channels': 32,
 'residual': True,
 'conv_drop': 0.0,
 'mlp_dim': 128,
 'lr': 0.000330227306657115,
 'weight_decay': 1.9723511474610607e-06}

In [8]:
conv_kernel = params["conv_kernel"]
bottleneck_dim = params["max_channels"]
pool = params["pool_op"]

pool_op = nn.AvgPool1d if pool == "avg" else nn.MaxPool1d

config = UnetConfig(
	input_shape=(256),
	input_channels=3,
	conv_op=nn.Conv1d,
	norm_op=nn.InstanceNorm1d,
	non_lin=nn.ReLU,
	pool_op=pool_op,
	up_op=nn.ConvTranspose1d,
	starting_channels=params["starting_channels"],
	max_channels=bottleneck_dim,
	conv_group=1,
	conv_kernel=conv_kernel,
	conv_padding=conv_kernel//2,
	pool_fact=2,
	deconv_group=1,
	deconv_padding=(0),
	deconv_kernel=(2),
	deconv_stride=(2),
	residual=params["residual"],
	conv_pdrop=params["conv_drop"]
)

classifier = BottleNeckClassifier((bottleneck_dim,params["mlp_dim"]),pool="max")

unet = Unet(config,classifier)
unet.to("cuda")

train_split = 3*[["train","test"]] +  3*[["train"]] + 3*[["train","test"]] 
test_split = + 3*[[]]+ 3* [["test"]] + 3*[[]]

lr = params["lr"]
weight_decay = params["weight_decay"]


In [9]:
clf = DeepClassifier(unet,
					 dataset,
					 train_split,
					 test_split,
					 length=2.5,
					 index_cutoff=256)

In [10]:
accuracies = clf.k_fold_cv(fabric=FABRIC,
			  k=3,
			  n=9,
			  lr=lr,
			  weight_decay=weight_decay,
			  verbose=False,
			  leave_out=False)

Finished Training
---
reached an accuracy of 74.55540355677155
---
Finished Training
---
reached an accuracy of 58.88324873096447
---
Finished Training
---
reached an accuracy of 70.3601108033241
---


- 73% 9-fold CV for 1s